# Tune a language model to generate responses to writing prompts
> Optimise a language model to produce responses to reddit writing prompts using a dataset of writing prompt response comparisons

## Setup experiment

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# hide
import sys
sys.path.append("..")

In [3]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-50fa6b69-6a5b-2dc7-2ac4-59c594b573af)


In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
!pip install wandb
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install sentencepiece

In [6]:
import torch
import wandb
import time
import os
from tqdm.auto import tqdm
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.optim import Adam
import torch
import collections
import time
import random
tqdm.pandas()

from datasets import load_dataset, ClassLabel, load_metric

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForPreTraining
from transformers import top_k_top_p_filtering
from torch import nn
from torch.nn import Identity
import torch.nn.functional as F
import torch

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, AdamW, get_scheduler, AutoModelForCausalLM, GPT2PreTrainedModel

from accelerate import Accelerator

# from trl_custom.model_value_head import LMHeadWithValueModel, respond_to_batch
# from trl_custom.ppo import PPOTrainer

In [7]:
def flatten_dict(nested, sep='/'):
    """Flatten dictionary and concatenate nested keys with separator."""
    def rec(nest, prefix, into):
        for k, v in nest.items():
            if sep in k:
                raise ValueError(f"separator '{sep}' not allowed to be in key '{k}'")
            if isinstance(v, collections.Mapping):
                rec(v, prefix + k + sep, into)
            else:
                into[prefix + k] = v
    flat = {}
    rec(nested, '', flat)
    return flat

def stack_dicts(stats_dicts):
    """Stack the values of a dict."""
    results = dict()
    for k in stats_dicts[0]:
        stats_list = [torch.flatten(d[k]) for d in stats_dicts]
        results[k] = torch.stack(stats_list)
    return results

def add_suffix(input_dict, suffix):
    """Add suffix to dict keys."""
    return dict((k + suffix, v) for k,v in input_dict.items())

# Cell

def pad_to_size(tensor, size, dim=1, padding=50256):
    """Pad tensor to size."""
    t_size = tensor.size()[dim]
    if t_size==size:
        return tensor
    else:
        return torch.nn.functional.pad(tensor, (0,size-t_size), 'constant', padding)

def logprobs_from_logits(logits, labels):
    """
    See: https://github.com/pytorch/pytorch/issues/563#issuecomment-330103591
    """
    logp = F.log_softmax(logits, dim=2)
    logpy = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logpy


def whiten(values, shift_mean=True):
    """Whiten values."""
    mean, var = torch.mean(values), torch.var(values)
    whitened = (values - mean) * torch.rsqrt(var + 1e-8)
    if not shift_mean:
        whitened += mean
    return whitened

def clip_by_value(x, tensor_min, tensor_max):
    """
    Tensor extenstion to torch.clamp
    https://github.com/pytorch/pytorch/issues/2793#issuecomment-428784713
    """
    clipped = torch.max(torch.min(x, tensor_max), tensor_min)
    return clipped

def entropy_from_logits(logits):
    """Calculate entropy from logits."""
    pd = torch.nn.functional.softmax(logits, dim=-1)
    entropy = torch.logsumexp(logits, axis=-1) - torch.sum(pd*logits, axis=-1)
    return entropy


def average_torch_dicts(list_of_dicts):
    """Average values of a list of dicts wiht torch tensors."""
    average_dict = dict()
    for key in list_of_dicts[0].keys():
        average_dict[key] = torch.mean(torch.stack([d[key] for d in list_of_dicts]), axis=0)
    return average_dict

def stats_to_np(stats_dict):
    """Cast all torch.tensors in dict to numpy arrays."""
    new_dict = dict()
    for k, v in stats_dict.items():
        if isinstance(v, torch.Tensor):
            new_dict[k] = v.detach().cpu().numpy()
        else:
            new_dict[k] = v
        if np.isscalar(new_dict[k]):
            new_dict[k] = float(new_dict[k])
    return new_dict

In [8]:
class ValueHead(nn.Module):
    """The ValueHead class implements a head for a language model that returns a scalar for each output token."""
    def __init__(self, config):
        super().__init__()
        self.summary_type = config.summary_type if hasattr(config, "summary_type") else "last"
        if self.summary_type == "attn":
            raise NotImplementedError

        self.summary = Identity()
        if hasattr(config, "summary_use_proj") and config.summary_use_proj:
            if hasattr(config, "summary_proj_to_labels") and config.summary_proj_to_labels and config.num_labels > 0:
                num_classes = config.num_labels
            else:
                num_classes = config.hidden_size
            self.summary = nn.Linear(config.hidden_size, num_classes)

        self.activation = Identity()
        if hasattr(config, "summary_activation") and config.summary_activation == "tanh":
            self.activation = nn.Tanh()

        self.first_dropout = Identity()
        if hasattr(config, "summary_first_dropout") and config.summary_first_dropout > 0:
            self.first_dropout = nn.Dropout(config.summary_first_dropout)

        self.last_dropout = Identity()
        if hasattr(config, "summary_last_dropout") and config.summary_last_dropout > 0:
            self.last_dropout = nn.Dropout(config.summary_last_dropout)

        self.flatten = nn.Flatten()

    def forward(self, hidden_states, cls_index=None):
        output = hidden_states
        output = self.first_dropout(output)
        output = self.summary(output)
        output = self.activation(output)
        output = self.last_dropout(output)

        return output

# Cell

class LMHeadWithValueModel(GPT2PreTrainedModel):
    """The LMHeadWithValueModel class implements a language model with a secondary, scalar head."""
    def __init__(self, config):
        super().__init__(config)
        config.num_labels = 1
        self.transformer = AutoModel.from_config(config)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.v_head = ValueHead(config)

        self.init_weights()
    
    def get_output_embeddings(self):
        return self.lm_head

    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        mc_token_ids=None,
        lm_labels=None,
        mc_labels=None,
    ):
       
        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        hidden_states = transformer_outputs[0]

        lm_logits = self.lm_head(hidden_states)
        value = self.v_head(hidden_states).squeeze(-1)

        outputs = (lm_logits,) + transformer_outputs[1:] + (value,)
        
        return outputs

# Cell

def respond_to_batch(model, queries, txt_len=20, top_k=0, top_p=1.0):
    """Sample text from language model."""
    input_ids = queries
    for i in range(txt_len):
        # Get Logits
        outputs = model(input_ids)
        next_token_logits = outputs[0][:, -1, :]
        next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
        # Sample
        probs = F.softmax(next_token_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1).squeeze(1)
        input_ids = torch.cat([input_ids, next_token.unsqueeze(-1)], dim=-1)
    return input_ids[:, -txt_len:]

In [9]:
class AdaptiveKLController:
    """
    Adaptive KL controller described in the paper:
    https://arxiv.org/pdf/1909.08593.pdf
    """
    def __init__(self, init_kl_coef, target, horizon):
        self.value = init_kl_coef
        self.target = target
        self.horizon = horizon

    def update(self, current, n_steps):
        target = self.target
        proportional_error = np.clip(current / target - 1, -0.2, 0.2)
        mult = 1 + proportional_error * n_steps / self.horizon
        self.value *= mult

# Cell

class FixedKLController:
    """Fixed KL controller."""
    def __init__(self, kl_coef):
        self.value = kl_coef

    def update(self, current, n_steps):
        pass

# Cell

class PPOTrainer:
    """
    The PPO_trainer uses Proximal Policy Optimization to optimise language models.
    """

    default_params = {
        "lr": 1.41e-5,
        "adap_kl_ctrl": True,
        "init_kl_coef":0.2,
        "target": 6,
        "horizon":10000,
        "gamma":1,
        "lam":0.95,
        "cliprange": .2,
        "cliprange_value":.2,
        "vf_coef":.1,
        "batch_size": 256,
        "forward_batch_size": 16,
        "ppo_epochs": 4,
    }

    def __init__(self, policy_model, ref_model, value_model, **ppo_params):
        """
        Initialize PPOTrainer.
        Args:
            model (torch.model): Hugging Face transformer model with value head
            ref_model (torch.model): Hugging Face transformer reference model used for KL penalty
            ppo_params (dict or None): PPO parameters for training. Can include following keys:
                'lr' (float): Adam learning rate, default: 1.41e-5
                'batch_size' (int): Number of samples per optimisation step, default: 256
                'forward_batch_size' (int): Number of samples forward passed through model at a time, default: 16
                'ppo_epochs' (int): Number of optimisation epochs per batch of samples, default: 4
                'gamma' (float)): Gamma parameter for advantage calculation, default: 1.
                'lam' (float): Lambda parameter for advantage calcualation, default: 0.95
                'cliprange_value' (float): Range for clipping values in loss calculation, default: 0.2
                'cliprange' (float): Range for clipping in PPO policy gradient loss, default: 0.2
                'vf_coef' (float): Scaling factor for value loss, default: 0.1
                'adap_kl_ctrl' (bool): Use adaptive KL control, otherwise linear, default: True
                'init_kl_coef' (float): Initial KL penalty coefficient (used for adaptive and linear control), default: 0.2
                'target' (float): Target KL value for adaptive KL control, default: 6.0
                'horizon' (float): Horizon for adaptive KL control, default: 10000
        """
        self.ppo_params = self.default_params
        self.ppo_params.update(ppo_params)

        self.ref_model = ref_model
        self.policy_model = policy_model
        self.value_model = value_model
        self.policy_optimizer = Adam(policy_model.parameters(), lr=self.ppo_params['lr'])
        self.value_optimizer = Adam(value_model.parameters(), lr=self.ppo_params['lr'])

        if self.ppo_params['adap_kl_ctrl']:
            self.kl_ctl = AdaptiveKLController(self.ppo_params['init_kl_coef'],
                                               self.ppo_params['target'],
                                               self.ppo_params['horizon'])
        else:
            self.kl_ctl = FixedKLController(self.ppo_params['init_kl_coef'])


    def step(self, query, response, scores):
        """
        Run a PPO optimisation step.
        args:
            query (torch.tensor): tensor containing the encoded queries, shape [batch_size, query_length]
            response (torch.tensor): tensor containing the encoded responses, shape [batch_size, response_length]
            scores (torch.tensor): tensor containing the scores, shape [batch_size]
        returns:
            train_stats (dict): a summary of the training statistics
        """

        bs = self.ppo_params['batch_size']
        timing = dict()
        t0 = time.time()

        gen_len = response.shape[1]
        model_input = torch.cat((query, response), axis=1)

        t = time.time()
        logprobs, ref_logprobs, values = self.batched_forward_pass(model_input, gen_len)
        timing['time/ppo/forward_pass'] = time.time()-t

        t = time.time()
        rewards, non_score_reward, kl_coef = self.compute_rewards(scores, logprobs, ref_logprobs)
        timing['time/ppo/compute_rewards'] = time.time()-t

        t = time.time()
        all_stats = []
        idxs = list(range(bs))
        for _ in range(self.ppo_params['ppo_epochs']):
            random.shuffle(idxs)
            for i in range(bs):
                idx = idxs[i]
                train_stats = self.train_minibatch(logprobs[idx:idx+1], values[idx:idx+1],
                                                   rewards[idx:idx+1],
                                                   response[idx:idx+1], model_input[idx:idx+1])
                all_stats.append(train_stats)
        timing['time/ppo/optimize_step'] = time.time()-t

        t = time.time()
        train_stats = stack_dicts(all_stats)

        # reshape advantages/ratios such that they are not averaged.
        train_stats['policy/advantages'] = torch.flatten(train_stats['policy/advantages']).unsqueeze(0)
        train_stats['policy/ratio'] = torch.flatten(train_stats['policy/ratio']).unsqueeze(0)

        stats = self.record_step_stats(scores=scores, logprobs=logprobs, ref_logprobs=ref_logprobs,
                                       non_score_reward=non_score_reward, train_stats=train_stats,
                                       kl_coef=kl_coef)
        stats = stats_to_np(stats)
        timing['time/ppo/calc_stats'] = time.time()-t

        self.kl_ctl.update(stats['objective/kl'], self.ppo_params['batch_size'])

        timing['time/ppo/total'] = time.time()-t0
        stats.update(timing)
        return stats

    def batched_forward_pass(self, model_input, gen_len):
        """Calculate model outputs in multiple batches."""
        bs = self.ppo_params['batch_size']
        fbs = self.ppo_params['forward_batch_size']
        logprobs = []
        ref_logprobs = []
        values = []

        for i in range(int(self.ppo_params['batch_size']/fbs)):
            m_input = model_input[i*fbs:(i+1)*fbs]
            logits, _, _ = self.policy_model(m_input)
            _, _, v = self.value_model(m_input)
            ref_logits, _, _ = self.ref_model(m_input)

            values.append(v[:, -gen_len-1:-1].detach())
            logprobs.append(logprobs_from_logits(logits[:,:-1,:], m_input[:,1:])[:, -gen_len:].detach())
            ref_logprobs.append(logprobs_from_logits(ref_logits[:,:-1,:], m_input[:,1:])[:, -gen_len:].detach())

        return torch.cat(logprobs), torch.cat(ref_logprobs), torch.cat(values)

    def train_minibatch(self, logprobs, values, rewards, response, model_input):
        """Train one PPO minibatch"""
        loss_p, train_stats  = self.loss_policy(logprobs, values, rewards, response, model_input)
        loss_v = self.loss_value(values, rewards, response, model_input)
        self.policy_optimizer.zero_grad()
        self.value_optimizer.zero_grad()
        loss_p.backward()
        loss_v.backward()
        self.policy_optimizer.step()
        self.value_optimizer.step()
        return train_stats

    def compute_rewards(self, scores, logprobs, ref_logprobs):
        """Compute per token rewards from scores and KL-penalty."""
        kl = logprobs - ref_logprobs
        non_score_reward = -self.kl_ctl.value * kl
        rewards = non_score_reward.clone().detach()
        rewards[:, -1] += scores
        return rewards, non_score_reward, self.kl_ctl.value

    def loss_value(self, values, rewards, response, model_input):
        """Calculate value loss"""
        lastgaelam = 0
        advantages_reversed = []
        gen_len = response.shape[1]

        for t in reversed(range(gen_len)):
            nextvalues = values[:, t + 1] if t < gen_len - 1 else 0.0
            delta = rewards[:, t] + self.ppo_params['gamma'] * nextvalues - values[:, t]
            lastgaelam = delta + self.ppo_params['gamma'] * self.ppo_params['lam'] * lastgaelam
            advantages_reversed.append(lastgaelam)
        advantages = torch.stack(advantages_reversed[::-1]).transpose(0, 1)

        returns = advantages + values
        advantages = whiten(advantages)
        advantages = advantages.detach()

        logits, _, _ = self.policy_model(model_input)
        _, _, vpred = self.value_model(model_input)

        logprob = logprobs_from_logits(logits[:,:-1,:], model_input[:, 1:])

        #only the generation part of the values/logprobs is needed
        logprob, vpred = logprob[:, -gen_len:], vpred[:,-gen_len-1:-1]

        vpredclipped = clip_by_value(vpred,
                                     values - self.ppo_params["cliprange_value"],
                                     values + self.ppo_params["cliprange_value"])

        vf_losses1 = (vpred - returns)**2
        vf_losses2 = (vpredclipped - returns)**2
        vf_loss = .5 * torch.mean(torch.max(vf_losses1, vf_losses2))

        return self.ppo_params['vf_coef'] * vf_loss

    def loss_policy(self, old_logprobs, values, rewards, response, model_input):
        """Calculate policy loss."""
        lastgaelam = 0
        advantages_reversed = []
        gen_len = response.shape[1]

        for t in reversed(range(gen_len)):
            nextvalues = values[:, t + 1] if t < gen_len - 1 else 0.0
            delta = rewards[:, t] + self.ppo_params['gamma'] * nextvalues - values[:, t]
            lastgaelam = delta + self.ppo_params['gamma'] * self.ppo_params['lam'] * lastgaelam
            advantages_reversed.append(lastgaelam)
        advantages = torch.stack(advantages_reversed[::-1]).transpose(0, 1)

        returns = advantages + values
        advantages = whiten(advantages)
        advantages = advantages.detach()

        logits, _, _ = self.policy_model(model_input)
        _, _, vpred = self.value_model(model_input)

        logprob = logprobs_from_logits(logits[:,:-1,:], model_input[:, 1:])

        #only the generation part of the values/logprobs is needed
        logprob, vpred = logprob[:, -gen_len:], vpred[:,-gen_len-1:-1]

        vpredclipped = clip_by_value(vpred,
                                     values - self.ppo_params["cliprange_value"],
                                     values + self.ppo_params["cliprange_value"])

        vf_losses1 = (vpred - returns)**2
        vf_losses2 = (vpredclipped - returns)**2
        vf_loss = .5 * torch.mean(torch.max(vf_losses1, vf_losses2))
        vf_clipfrac =  torch.mean(torch.gt(vf_losses2, vf_losses1).double())

        ratio = torch.exp(logprob - old_logprobs)

        pg_losses = -advantages * ratio
        pg_losses2 = -advantages * torch.clamp(ratio,
                                               1.0 - self.ppo_params['cliprange'],
                                               1.0 + self.ppo_params['cliprange'])

        pg_loss = torch.mean(torch.max(pg_losses, pg_losses2))
        pg_clipfrac = torch.mean(torch.gt(pg_losses2, pg_losses).double())

        entropy = torch.mean(entropy_from_logits(logits))
        approxkl = .5 * torch.mean((logprob - old_logprobs)**2)
        policykl = torch.mean(logprob - old_logprobs)
        return_mean, return_var = torch.mean(returns), torch.var(returns)
        value_mean, value_var = torch.mean(values), torch.var(values)

        stats = dict(
            loss=dict(policy=pg_loss, value=vf_loss),
            policy=dict(entropy=entropy, approxkl=approxkl,policykl=policykl, clipfrac=pg_clipfrac,
                        advantages=advantages, advantages_mean=torch.mean(advantages), ratio=ratio),
            returns=dict(mean=return_mean, var=return_var),
            val=dict(vpred=torch.mean(vpred), error=torch.mean((vpred - returns) ** 2),
                     clipfrac=vf_clipfrac, mean=value_mean, var=value_var),
        )
        return pg_loss, flatten_dict(stats)


    def record_step_stats(self, kl_coef, **data):
        """Record training step statistics."""
        kl = data['logprobs'] - data['ref_logprobs']
        mean_kl = torch.mean(torch.sum(kl, axis=-1))
        mean_entropy = torch.mean(torch.sum(-data['logprobs'], axis=1))
        mean_non_score_reward =torch.mean(torch.sum(data['non_score_reward'], axis=1))
        stats = {
            'objective/kl': mean_kl,
            'objective/kl_dist': kl,
            'objective/logprobs': data['logprobs'],
            'objective/ref_logprobs': data['ref_logprobs'],
            'objective/kl_coef': kl_coef,
            'objective/entropy': mean_entropy,
            'ppo/mean_non_score_reward': mean_non_score_reward,
        }

        for k, v in data['train_stats'].items():
            stats[f'ppo/{k}'] = torch.mean(v, axis=0)
        stats['ppo/val/var_explained'] = 1 - stats['ppo/val/error'] / stats['ppo/returns/var']
        return stats

### Train Reward Model on Writing Prompt Response Comparisons

In [10]:
prompt_response_dataset = load_dataset("rewardsignal/reddit_writing_prompts", data_files="prompt_responses_full.csv", split='train[:80%]')

Using custom data configuration rewardsignal--reddit_writing_prompts-dd5d2a64487ab606
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


In [11]:
prompt_response_dataset.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'num_responses': Value(dtype='float64', id=None),
 'prompt': Value(dtype='string', id=None),
 'prompt_created_utc': Value(dtype='float64', id=None),
 'prompt_id': Value(dtype='string', id=None),
 'prompt_score': Value(dtype='float64', id=None),
 'response': Value(dtype='string', id=None),
 'response_children': Value(dtype='string', id=None),
 'response_created_utc': Value(dtype='float64', id=None),
 'response_id': Value(dtype='string', id=None),
 'response_rank': Value(dtype='float64', id=None),
 'response_score': Value(dtype='float64', id=None),
 'score_bin': Value(dtype='float64', id=None)}

In [12]:
# tokenizer_name = input()
# tokenizer_name = 'distilgpt2'
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
# prompt_prefix = "Writing Prompt: "
# response_prefix = "Response: "

# def preprocess_text_function(examples):
#   examples["prompt"] = [prompt.replace('[WP] ', prompt_prefix) for prompt in examples["prompt"]]
#   examples["response"] = [response_prefix + response for response in examples["response"]]
#   return tokenizer(examples['prompt'], examples['response'], truncation=True)

# tokenized_reward_dataset = prompt_response_dataset.map(preprocess_text_function, batched=True)

In [13]:
# tokenized_reward_dataset = tokenized_reward_dataset.remove_columns(['Unnamed: 0', 'prompt_id', 'prompt', 'prompt_score', 'prompt_created_utc', 'response_id', 'response', 'response_score', 'response_created_utc', 'num_responses', 'response_children', 'score_bin']
# )
# tokenized_reward_dataset = tokenized_reward_dataset.rename_column('response_rank', 'labels')
# def preprocess_labels_function(examples):
#   examples['labels'] = 1 if (examples['labels'] == 0) else 0
#   return examples
# tokenized_reward_dataset = tokenized_reward_dataset.map(preprocess_labels_function)
# tokenized_reward_dataset = tokenized_reward_dataset.cast_column('labels', ClassLabel(num_classes=2, names=['best', 'not-best'], names_file=None, id=None))
# tokenized_reward_dataset.set_format("torch")

In [14]:
# reward_train_dataset = tokenized_reward_dataset.shuffle(seed=42).select(range(3*len(tokenized_reward_dataset)//4))
# reward_eval_dataset = tokenized_reward_dataset.shuffle(seed=42).select(range(3*len(tokenized_reward_dataset)//4, len(tokenized_reward_dataset)))

In [15]:
# # model_name = input()
# tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# reward_model_name = tokenizer_name
# reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name, num_labels=2)
# reward_model.config.pad_token_id = reward_model.config.eos_token_id

# train_dataloader = DataLoader(
#     reward_train_dataset, shuffle=True, batch_size=4, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     reward_eval_dataset, batch_size=4, collate_fn=data_collator
# )

# optimizer = AdamW(reward_model.parameters(), lr=3e-5)
# accelerator = Accelerator()
# train_dataloader, eval_dataloader, reward_model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, reward_model, optimizer)
# num_epochs = 1
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )

# progress_bar = tqdm(range(num_training_steps))

In [16]:
!pip install huggingface_hub
!apt install git-lfs
from huggingface_hub import notebook_login
notebook_login()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [17]:
# reward_model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         outputs = reward_model(**batch)
#         loss = outputs.loss
#         accelerator.backward(loss)
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)
# reward_model.push_to_hub(tokenizer_name + "_reward_model_02", use_temp_dir=True)

In [18]:
# len(eval_dataloader)

In [19]:
# metric = load_metric("accuracy")
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# reward_model.to(device)
# reward_model.eval()
# count = 0
# for batch in eval_dataloader:
#     count += 1
#     batch = {k: v.to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = reward_model(**batch)

#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)
#     metric.add_batch(predictions=predictions, references=batch["labels"])
#     if (count % 1000 == 0):
#       print(count)

# metric.compute()

### Fine-tune LM on responding to writing prompts 

In [20]:
# tokenizer_name = input()
tokenizer_name = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
prompt_prefix = "Writing Prompt: "
response_prefix = "Response: "

def preprocess_text_function(examples):
  examples["prompt"] = [prompt.replace('[WP] ', prompt_prefix) for prompt in examples["prompt"]]
  examples["response"] = [response_prefix + response for response in examples["response"]]
  return tokenizer(examples['prompt'], examples['response'], truncation=True)

tokenized_prompt_response_dataset = prompt_response_dataset.map(preprocess_text_function, batched=True, remove_columns=['Unnamed: 0', 'prompt_id', 'prompt', 'prompt_score', 'prompt_created_utc', 'response_id', 'response', 'response_score', 'response_created_utc', 'num_responses', 'response_children', 'score_bin', 'response_rank'], num_proc=4)


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-5bc08506140d8b27.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-08bf676bb657ea28.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-a13ff98572a05dfb.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-12fa3a11c1a663cb.arrow


In [21]:
block_size = 512
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
tokenized_prompt_response_dataset = tokenized_prompt_response_dataset.map(group_texts, batched=True, batch_size = 1000, num_proc = 4)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-c95c481401b11fc4.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-c5840e979d00732e.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-33fcc7f14548838d.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/rewardsignal--reddit_writing_prompts-dd5d2a64487ab606/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-5af14d4e66eb4003.arrow


In [22]:
tokenizer.decode(tokenized_prompt_response_dataset[1]["input_ids"])

'�So… why did we let that dragon inside, exactly?” I asked.\n\n“Company policy, based on our past experiences,” he said. “The last woman whose emotional support dragon was denied entry, came back with her financial support lawyer.” He blew out a smoke ring. “Believe me when I say it is cheaper that way, and by a lot.”Writing Prompt: "Ma\'am you can\'t bring your emotional support dragon inside the restaurant."Response: "Well, why the hell not?!"\n\nThe host flicked her eyes up to meet the unaffected gaze of the giant blue beast, and then back down to the woman who wore the bracelet which assigned it as hers.\n\n"Ma\'am, this is a small restaurant, he won\'t fit inside," the host explained patiently, a bit astounded by the idea that this woman believed her enormous, full grown dragon would fit.\n\nThe lady huffed, then stammered, "Well...what about outside seating."\n\nThe host surveyed the sidewalk area shaded by an awning that flapped about in the slight breeze. The tables were full b

In [23]:
tokenized_prompt_response_dataset.set_format("torch")

In [24]:
# supervised_train_dataset = tokenized_prompt_response_dataset.shuffle(seed=42).select(range(3*len(tokenized_prompt_response_dataset)//4))
# supervised_eval_dataset = tokenized_prompt_response_dataset.shuffle(seed=42).select(range(3*len(tokenized_prompt_response_dataset)//4, len(tokenized_prompt_response_dataset)))

In [25]:
# model_name = input()
# from transformers import AutoModelForCausalLM
# tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# supervised_model_name = tokenizer_name
# supervised_model = AutoModelForCausalLM.from_pretrained(supervised_model_name, num_labels=2)

# train_dataloader = DataLoader(
#     supervised_train_dataset, shuffle=True, batch_size=4, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     supervised_eval_dataset, batch_size=4, collate_fn=data_collator
# )

# optimizer = AdamW(supervised_model.parameters(), lr=3e-5)
# accelerator = Accelerator()
# train_dataloader, eval_dataloader, supervised_model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, supervised_model, optimizer)
# num_epochs = 1
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )

# progress_bar = tqdm(range(num_training_steps))

In [26]:
# supervised_model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         outputs = supervised_model(**batch)
#         loss = outputs.loss
#         accelerator.backward(loss)
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)
# supervised_model.push_to_hub(tokenizer_name + "_supervised_model_01", use_temp_dir=True)

In [27]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# saved_model = AutoModelForCausalLM.from_pretrained("anshr/distilgpt2_supervised_model_01")
# saved_model.to(device)
# count = 0
# for batch in eval_dataloader:
#     count += 1
#     batch = {k: v.to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = saved_model.generate(**batch, max_length=512, min_length = 200)
#         print(tokenizer.batch_decode(outputs, max_length = 512))
#     if count == 10: break



## Load models

### Load Reward Model
We load a DistilGPT2 classifier fine-tuned on the writing prompt dataset to predict whether a response is the best response (as judged by karma) or not.

In [28]:
reward_model = AutoModelForSequenceClassification.from_pretrained("anshr/distilgpt2_reward_model_02")
reward_model_tokenizer = AutoTokenizer.from_pretrained("anshr/distilgpt2_reward_model_02")

The model outputs are the logits for the not-best and best classes. We will use the logits for the best class as a reward signal for the language model.

### Load pre-trained DistilGPT2 language models

We load the DistilGPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [29]:
policy_model = LMHeadWithValueModel.from_pretrained("anshr/distilgpt2_supervised_model_01")
policy_model_ref = LMHeadWithValueModel.from_pretrained("anshr/distilgpt2_supervised_model_01")
value_model = LMHeadWithValueModel.from_pretrained("anshr/distilgpt2_supervised_model_01")
policy_tokenizer = AutoTokenizer.from_pretrained("anshr/distilgpt2_supervised_model_01")

Some weights of LMHeadWithValueModel were not initialized from the model checkpoint at anshr/distilgpt2_supervised_model_01 and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of LMHeadWithValueModel were not initialized from the model checkpoint at anshr/distilgpt2_supervised_model_01 and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of LMHeadWithValueModel were not initialized from the model checkpoint at anshr/distilgpt2_supervised_model_01 and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Move models to GPU

If `cuda` is available move the computations to the GPU.

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
_ = reward_model.to(device)
_ = policy_model.to(device)
_ = policy_model_ref.to(device)
_ = value_model.to(device)

In [32]:
config = {
    "steps": 25600,
    "batch_size": 16,
    "forward_batch_size": 4,
    "ppo_epochs": 4,   
    "txt_in_len": 20,
    "txt_out_len": 100,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
}

In [33]:
def build_reward_batch_from_txt(text_list, tokenizer, device):
  # tokenize
    tensors = [tokenizer.encode(txt, return_tensors="pt").to(device) for txt in text_list]
    
    # find max length to pad to
    max_len = max([t.size()[1] for t in tensors])
    
    # get padded tensors and attention masks
    padded_tensors = []
    attention_masks = []
    for tensor in tensors:
        attention_mask = torch.ones(tensor.size(), device=device)
        padded_tensors.append(pad_to_size(tensor, max_len, padding=0))
        attention_masks.append(pad_to_size(attention_mask, max_len, padding=0))
    
    # stack all tensors
    padded_tensors = torch.cat(padded_tensors)
    attention_masks = torch.cat(attention_masks)  
    
    return padded_tensors, attention_masks

## Optimize model

**Steps**

The training loop consists of the following steps:
1. Get a batch of queries
2. Get the query responses from the policy
3. Join query and responses and tokenize for BERT analysis
4. Get sentiments for query/responses from BERT
5. Optimize policy with PPO using the (query, response, reward) triplet
6. Log all the training statistics

**Forward batching**

Since the models can be fairly big and we want to rollout large PPO batches this can lead to out-of-memory errors when doing the forward passes for text generation and sentiment analysis. We introduce the parameter `forward_batch_size` to split the forward passes into smaller batches. Although this hurts performance a little this is neglectible compared to the computations of the backward passes when optimizing the model. The same parameter is used in the `PPOTrainer` when doing forward passes. The `batch_size` should multiple of `forward_batch_size`.

**Training time**

This step takes **~2h** on a P6000 GPU with the above specified settings.

In [34]:
ppo_trainer = PPOTrainer(policy_model, policy_model_ref, value_model, **config)
fbs = config['forward_batch_size']

for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    
    #### get a batch from the dataset
    df_batch = tokenized_prompt_response_dataset.shuffle().select(range(config["batch_size"]))
    game_data['query'] = [policy_tokenizer.decode(_) for _ in df_batch['input_ids'][:, :config["txt_in_len"]].to(device)]
    query_tensors = df_batch['input_ids'][:, :config["txt_in_len"]].to(device)
    
    #### get response from gpt2
    t = time.time()
    total_length = config['txt_in_len']+config['txt_out_len']
    response_tensors = []
    for i in range(int(config['batch_size']/fbs)):
        response  = respond_to_batch(policy_model, query_tensors[i*fbs:(i+1)*fbs],
                                     txt_len=config['txt_out_len'])
        response_tensors.append(response)
    response_tensors = torch.cat(response_tensors)
    game_data['response'] = [policy_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])]
    timing['time/get_response'] = time.time()-t

    #### tokenize text for sentiment analysis
    t = time.time()
    texts = [q + r for q,r in zip(game_data['query'], game_data['response'])]
    reward_inputs, attention_masks = build_reward_batch_from_txt(texts, reward_model_tokenizer, device)    
    timing['time/build_input_sentiment'] = time.time()-t

    #### get sentiment score
    t = time.time()
    rewards = []
    for i in range(int(config['batch_size']/fbs)):
        res = reward_model.forward(input_ids=reward_inputs[i*fbs:(i+1)*fbs],
                                      attention_mask=attention_masks[i*fbs:(i+1)*fbs])[0][:, 1].detach()
        rewards.append(res)
    rewards = torch.cat(rewards)
    timing['time/get_sentiment_preds'] = time.time()-t
    
    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    timing['time/optimization'] = time.time()-t
     
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]

  0%|          | 0/1600 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  import sys


## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `gpt2_model_ref` to compare the tuned model `gpt2_model` against the model before optimisation.

In [ ]:
#### get a batch from the dataset
bs = 16
game_data = dict()
df_batch = tokenized_prompt_response_dataset.shuffle().select(bs))
game_data['query'] = [policy_tokenizer.decode(_) for _ in df_batch['input_ids']]
query_tensors = df_batch['input_ids'].to(device)

#### get response from gpt2 and gpt2_ref
total_length = config['txt_in_len']+config['txt_out_len']
response_tensors_ref  = respond_to_batch(gpt2_model_ref, query_tensors, txt_len=config['txt_out_len'])
game_data['response (before)'] = [gpt2_tokenizer.decode(response_tensors_ref[i, :]) for i in range(bs)]

response_tensors  = respond_to_batch(gpt2_model, query_tensors, txt_len=config['txt_out_len'])
game_data['response (after)'] = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q,r in zip(game_data['query'], game_data['response (before)'])]
sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
game_data['rewards (before)'] = rewards.cpu().numpy()

texts = [q + r for q,r in zip(game_data['query'], game_data['response (after)'])]
sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
game_data['rewards (after)'] = rewards.cpu().numpy()

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

,query,response (before),response (after),rewards (before),rewards (after)
0,That's what the title,said to me. All I had to do was hear one word...,"says, and this movie was very good,-I really ...",1.710360,4.207656
1,The female hot numbers on,"the Fonda get a saving grace here, you can st...",the DVD are beautiful. The music is extraordi...,1.585194,4.228330
2,In the film Kongwon,"(American Truffaut), a snake made of ash fall...",has a funky personality and makes the movies ...,-0.015413,2.537073
3,Let's see: what,"a blast from start to finish, female and male...",happens on that later?<|endoftext|> This film...,3.278678,4.061680
4,Really? Is this necessary,for Sith philosophy combined with missionary ...,"? Watched this too, but it's what made it so b...",-2.509291,3.025524
5,During a sleepless night,"Keira Knightley has besieged a monastery, tro...",", I found this Parker Bros. film. The acting i...",1.341672,3.933312
6,My wife spotted this film,"almost halfway through second grade, and afte...",when I was younger and kept her P-1-her-p-,1.781651,1.456647
7,I love John's work,but I actually have to write language as in w...,and I hereby recommend this film. I am really...,-1.904006,4.159198
8,I's a big struggle,to see anyone who acts in that way. by Jim Th...,", but overall I'm happy with the changes even ...",-1.595925,2.651260
9,i liked this film a,lot Paul Verhoeven The other actor does what ...,lot recipe. It was probably one of the best m...,4.314877,4.346435


Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [ ]:
print('mean:')
display(df_results.mean())
print()
print('median:')
display(df_results.median())

mean:


rewards (before)    0.211196
rewards (after)     3.068743
dtype: float64


median:


rewards (before)    0.296938
rewards (after)     3.997496
dtype: float64

## Save model
Finally, we save the model to disk for later usage.

In [ ]:
os.makedirs('gpt2-imdb-pos')
gpt2_model.save_pretrained('gpt2-imdb-pos')
gpt2_tokenizer.save_pretrained('gpt2-imdb-pos')

('gpt2-imdb-pos/vocab.json',
 'gpt2-imdb-pos/merges.txt',
 'gpt2-imdb-pos/special_tokens_map.json',
 'gpt2-imdb-pos/added_tokens.json')